In [1]:
import numpy as np
import pandas as pd

import astropy.units as u
import astropy.constants as c

from scipy.interpolate import LinearNDInterpolator

In [2]:
 # Grids of giants for different phases
# Each grid consists of the giant mass at the beginning of the phase, total duration of its giantphase
# and a list of masslosses evenly spaced between 0% of total duration to 100% of total duration in steps of 1%

RGB_grid_inter = pd.read_pickle('../grids/RGB_grid_inter')
HeB_grid_inter = pd.read_pickle('../grids/HeB_grid_inter')
AGB_grid_inter = pd.read_pickle('../grids/AGB_grid_inter')

tpAGB_grid_inter = pd.read_pickle('../grids/tpAGB_grid_inter')

In [3]:
RGB_Mdots = []
for i in range(len(RGB_grid_inter['Mdot'])):
    
    RGB_Mdots.append(np.array(RGB_grid_inter['Mdot'][i]))
    
RGB_Mdots = np.array(RGB_Mdots)
RGB_Mass0 = np.array(RGB_grid_inter['Mass0'])

In [4]:
HeB_Mdots = []
for i in range(len(HeB_grid_inter['Mdot'])):
    
    HeB_Mdots.append(np.array(HeB_grid_inter['Mdot'][i]))
    
HeB_Mdots = np.array(HeB_Mdots)
HeB_Mass0 = np.array(HeB_grid_inter['Mass0'])

In [5]:
AGB_Mdots = []
for i in range(len(AGB_grid_inter['Mdot'])):
    
    AGB_Mdots.append(np.array(AGB_grid_inter['Mdot'][i]))
    
AGB_Mdots = np.array(AGB_Mdots)
AGB_Mass0 = np.array(AGB_grid_inter['Mass0'])

In [6]:
tpAGB_Mdots = []
for i in range(len(tpAGB_grid_inter['Mdot'])):
    
    tpAGB_Mdots.append(np.array(tpAGB_grid_inter['Mdot'][i]))
    
tpAGB_Mdots = np.array(tpAGB_Mdots)
tpAGB_Mass0 = np.array(tpAGB_grid_inter['Mass0'])

***

In [7]:
RGB_Mass0 = np.array(RGB_grid_inter['Mass0'])
RGB_TimeTot = np.array(RGB_grid_inter['TimeTot'])
RGB_stages = np.linspace(0,1,101)

RGB_stage_mesh, RGB_Mass_mesh = np.meshgrid(RGB_stages, RGB_Mass0)


Mdots_RGB_interp = LinearNDInterpolator(list(zip(RGB_stage_mesh.flatten(), RGB_Mass_mesh.flatten())), RGB_Mdots.flatten())

In [8]:
HeB_Mass0 = np.array(HeB_grid_inter['Mass0'])
HeB_TimeTot = np.array(HeB_grid_inter['TimeTot'])
HeB_stages = np.linspace(0,1,101)

HeB_stage_mesh, HeB_Mass_mesh = np.meshgrid(HeB_stages, HeB_Mass0)


Mdots_HeB_interp = LinearNDInterpolator(list(zip(HeB_stage_mesh.flatten(), HeB_Mass_mesh.flatten())), HeB_Mdots.flatten())

In [9]:
AGB_Mass0 = np.array(AGB_grid_inter['Mass0'])
AGB_TimeTot = np.array(AGB_grid_inter['TimeTot'])
AGB_stages = np.linspace(0,1,101)

AGB_stage_mesh, AGB_Mass_mesh = np.meshgrid(AGB_stages, AGB_Mass0)


Mdots_AGB_interp = LinearNDInterpolator(list(zip(AGB_stage_mesh.flatten(), AGB_Mass_mesh.flatten())), AGB_Mdots.flatten())

In [10]:
tpAGB_Mass0 = np.array(tpAGB_grid_inter['Mass0'])
tpAGB_TimeTot = np.array(tpAGB_grid_inter['TimeTot'])
tpAGB_stages = np.linspace(0,1,101)

tpAGB_stage_mesh, tpAGB_Mass_mesh = np.meshgrid(tpAGB_stages, tpAGB_Mass0)


Mdots_tpAGB_interp = LinearNDInterpolator(list(zip(tpAGB_stage_mesh.flatten(), tpAGB_Mass_mesh.flatten())), tpAGB_Mdots.flatten())

In [11]:
def interpWindLossALL_t(Type_G, M_G, t):
        
    ALL_grid_Mass0 = np.array([np.array(RGB_grid_inter['Mass0']), np.array(HeB_grid_inter['Mass0']), np.array(AGB_grid_inter['Mass0']), np.array(tpAGB_grid_inter['Mass0'])])[Type_G - 3]
    ALL_grid_TimeTot = np.array([np.array(RGB_grid_inter['TimeTot']), np.array(HeB_grid_inter['TimeTot']), np.array(AGB_grid_inter['TimeTot']), np.array(tpAGB_grid_inter['TimeTot'])])[Type_G - 3]
        
    ############################
    
    TimeTot_RGB = np.interp(M_G.to(u.Msun).value, RGB_Mass0, RGB_TimeTot)
    TimeTot_HeB = np.interp(M_G.to(u.Msun).value, HeB_Mass0, HeB_TimeTot)
    TimeTot_AGB = np.interp(M_G.to(u.Msun).value, AGB_Mass0, AGB_TimeTot)
    TimeTot_tpAGB = np.interp(M_G.to(u.Msun).value, tpAGB_Mass0, tpAGB_TimeTot)
    
    TimeTot_all = np.array([TimeTot_RGB,TimeTot_HeB,TimeTot_AGB,TimeTot_tpAGB])
    
    TimeTot_G = TimeTot_all[Type_G - 3, np.arange(len(M_G))] * u.Myr
            
    ############################

    stage = (t/TimeTot_G).to(1).value
            
    stage[stage > 1] = 1
        
    ############################
    
    Mdots_RGB = Mdots_RGB_interp(stage,M_G)
    
    ############################

    Mdots_HeB = Mdots_HeB_interp(stage,M_G)
    
    ############################

    Mdots_AGB = Mdots_AGB_interp(stage,M_G)
    
    ############################

    Mdots_tpAGB = Mdots_tpAGB_interp(stage,M_G)
    
    ############################
    
    Mdots_all = np.array([Mdots_RGB,Mdots_HeB,Mdots_AGB,Mdots_tpAGB])
    
    ############################

    Mdots = Mdots_all[Type_G - 3, :, np.arange(len(M_G))]    
    
    return Mdots * u.Msun / u.yr

## Accretion rate

In [12]:
# accretion rate
def Mdot_acc(M_WD, a, v_orb, Mdot_G, R_G, M_G):

    # for circles r = a, used in v_w
    
    vw = v_w(a, R_G, M_G)
    
    eps = 3/2
    v = v_orb / vw
    
    Mdot_WD = (c.G * M_WD / vw**2)**2 * (eps / (2 * a**2)) / (1 + v**2)**(3/2) * Mdot_G
    
    return Mdot_WD.to(u.Msun / u.yr) # Msun yr^-1

# accumulation rate
def Mdot_accum(Mdotacc, M_WD):

    alpha = alpha_H(Mdotacc, M_WD)
        
    Mdotaccum = alpha * Mdotacc
    
    return Mdotaccum.to(u.Msun / u.yr) # Msun yr^-1

In [13]:
def v_w(r, R_G, M_G):
    
    return (alpha_w(r, R_G) * v_inf(M_G, R_G)).to(u.km / u.s)

def alpha_w(r, R_d):
    
    return (0.04 * (r/R_d)**2 / (1 + 0.04 * (r/R_d)**2)).to(1)

def v_inf(M, R):
    
    return (1/2 * v_esc(M, R)).to(u.km / u.s)

def v_esc(M, R):
    
    return np.sqrt(2 * c.G * M / R).to(u.km / u.s)

## Accumulation Efficiency

In [14]:
# is this correct?
def Mdot_st(M_WD):
    
    return 10**( -9.31 + 4.12 * M_WD.to(u.Msun).value -1.42 * (M_WD.to(u.Msun).value)**2 ) * u.Msun / u.yr

# still not sure whether the next should be in g or Msun
def Mdot_ws(M_WD):
            
    Mdot_ws_low  = 10**(-11.01 + 6 * M_WD.to(u.Msun).value - 1.90 * (M_WD.to(u.Msun).value)**2) * u.Msun / u.yr
    
    Mdot_ws_high = np.ones(len(M_WD)) * 10**(-7.0) * u.Msun / u.yr
    
    Mdot_ws_both = np.array([Mdot_ws_high, Mdot_ws_low]) # I changed this, so maybe check again later
    
    Mdot_ws_ind = (M_WD <= 1 * u.Msun) + 0
    
    Mdot = np.zeros(len(M_WD))
    
    for i in range(len(M_WD)):
        Mdot[i] = Mdot_ws_both[Mdot_ws_ind[i],i]
        
    return Mdot * u.Msun / u.yr

In [15]:
# normally we need the efficiency alpha that depends on weak/strong
def isStable(M_WD, Mdot_acc):
        
    return Mdot_acc > Mdot_st(M_WD)
    
def isWeak(M_WD, Mdot_acc):
    
    return Mdot_acc > Mdot_ws(M_WD)

In [16]:
# Accumulation efficiency

def alpha_H(Mdot_acc, M_WD):
    
    alphas_weak   = alpha_H_weak(Mdot_acc)
    alphas_strong = alpha_H_strong(M_WD)

    alphas_both = np.array([alphas_strong, alphas_weak])
    
    alphas_H = np.zeros(len(Mdot_acc))
        
    alpha_both_ind = isWeak(M_WD, Mdot_acc) + 0 # weak get index 1 and strong index 0 (True + 0 = 1, False + 0 = 0)
        
    for i in range(len(Mdot_acc)):
        alphas_H[i] = alphas_both[alpha_both_ind[i],i]
    
    return alphas_H


def alpha_H_weak(Mdot_acc):
    
    log_Mdot = np.log10(Mdot_acc.to(u.Msun / u.yr).value)
    
    alpha_low  = (-4.39 - 1.48 * log_Mdot - 0.1  * log_Mdot**2)
    alpha_high = (11.66 + 4.56 * log_Mdot + 0.45 * log_Mdot**2)
    
    alpha_both = np.array([alpha_low, alpha_high])
    
    #possibly quite slow
    alphas = np.zeros(len(Mdot_acc))

    alpha_both_ind = (log_Mdot >= -6.36) + 0
    
    for i in range(len(Mdot_acc)):
        alphas[i] = alpha_both[alpha_both_ind[i],i]
    
    return alphas
    
def alpha_H_strong(M_WD):
    
    return (-0.1391 + 0.7548 * M_WD.to(u.Msun).value - 1.0124 * (M_WD.to(u.Msun).value)**2 + 0.4739 * (M_WD.to(u.Msun).value)**3)

## Critical Ignition Mass

In [17]:
# in solar masses
def ign_mass(M_WD):
    
    R_zt_WD = R_zeroTemp_WD(M_WD)
    
    return 2 * 10**(-6) * (M_WD.to(u.g).value / R_zt_WD.to(u.cm).value**4)**(-0.8) * u.Msun

def R_zeroTemp_WD(M_WD):
    
    M_ch = 1.433 * u.Msun
    
    return (1.12 * 10**(-2) * u.Rsun * ( (M_WD/M_ch).to(1)**(-2/3) - (M_WD/M_ch).to(1)**(2/3) )**(1/2)).to(u.cm)

## Lifetime SyS phase

In [18]:
# in year
def t_on_fun(Mdot_acc, M_WD, R_WD):
    
    L_WD = L_H(M_WD)
    
    return (6.9 * 10**10 * (alpha_H(Mdot_acc, M_WD) * ign_mass(M_WD).to(u.Msun).value / L_WD.to(u.Lsun).value) * u.yr).to(u.Myr)

def t_cool_fun(M_WD):
    
    L_WD = L_H(M_WD)
    
    return ((10 / L_WD.to(u.Lsun).value)**(-1/1.14) * u.yr).to(u.Myr)

## Luminosity

In [19]:
def L_Sy(M_WD, R_WD, Mdotaccum):
    
    return L_H(M_WD) + L_grav(M_WD, R_WD, Mdotaccum)

# I am assuming that the M_core comes from the M_WD without the accreted mass but this should be checked

def L_H(M_WD):
    
    return 4.6 * 10**4 * (M_WD.to(u.Msun).value - 0.26) * u.Lsun

# Until I know how Lu got alpha = 0.1 I will take alpha = 0.15 from Yaron

def L_grav(M_WD, R_WD, Mdotaccum):
        
    return (0.15 * c.G * M_WD * Mdotaccum / R_WD).to(u.Lsun)